## Packages

In [ ]:
import pandas as pd
import numpy as np
import sqlalchemy 
from sqlalchemy import create_engine, text
from db_secrets import SQL_107

## Test Connection

In [ ]:
SQL_107()

In [ ]:
# Create an engine
engine = create_engine(SQL_107())

# Connect and fetch the top 100 rows
with engine.connect() as connection:
    result = connection.execute(text("SELECT TOP 100 * FROM [UEC_Aggregated_ALL]"))
    rows = result.fetchall()

# Display results
for row in rows:
    print(row)